In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [7]:
import os

print("Loading data...")
pdf_folder_path = "biblioteca/"
print(os.listdir(pdf_folder_path))

# Load multiple files
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]

print(loaders)

all_documents = []

for loader in loaders:
    print("Loading raw document..." + loader.file_path)
    raw_documents = loader.load()

    print("Splitting text...")
    text_splitter = RecursiveCharacterTextSplitter(
        #separator="\n\n",
        chunk_size=800,
        chunk_overlap=100,
        length_function=len,
    )
    documents = text_splitter.split_documents(raw_documents)
    all_documents.extend(documents)

Loading data...
['Cartilha LGPD.pdf', 'cartilha_lgpd_2021.pdf', 'ufs_cartilha_lgpd_.pdf']
[<langchain_community.document_loaders.pdf.UnstructuredPDFLoader object at 0x0000022BAB138C40>, <langchain_community.document_loaders.pdf.UnstructuredPDFLoader object at 0x0000022BAB1385B0>, <langchain_community.document_loaders.pdf.UnstructuredPDFLoader object at 0x0000022BAB138970>]
Loading raw document...biblioteca/Cartilha LGPD.pdf
Splitting text...
Loading raw document...biblioteca/cartilha_lgpd_2021.pdf
Splitting text...
Loading raw document...biblioteca/ufs_cartilha_lgpd_.pdf
Splitting text...


In [4]:
local_path = 'biblioteca/cartilha_lgpd_2021.pdf'

if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
else:
    print('Upload um arquivo')

In [5]:
type(data)#[0].page_content

list

### Vector Embeddings

In [4]:
!ollama list

NAME                             	ID          	SIZE  	MODIFIED          
cnmoro/mistral_7b_portuguese:q2_K	321345d18e55	2.7 GB	52 minutes ago   	
nomic-embed-text:latest          	0a109f422b47	274 MB	About an hour ago	


In [5]:
#!pip install --q chromadb
#!pip install --q langchain-text-splitters

In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [2]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, 
                                               chunk_overlap=100)

chunks = text_splitter.split_documents(data)

NameError: name 'data' is not defined

In [8]:
model_name = 'cnmoro/mistral_7b_portuguese:q2_K'

vector_db = Chroma.from_documents(
    documents = all_documents, #chunks, 
    embedding = OllamaEmbeddings(model=model_name, show_progress=True),
    collection_name='local-rag'
)

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 96/96 [04:52<00:00,  3.05s/it]


### Retrieval

In [9]:
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever

In [10]:
local_model = model_name
llm = ChatOllama(model=local_model)

In [11]:
QUERY_PROMPT = PromptTemplate(
    input_variables=['question'],
    template = """
    Você é um modelo de linguagem de IA. 
    Sua tarefa é gerar cinco diferentes versões 
    de uma questão dada pelo usuário para recuperar documentos relevantes de um vector database.
    Ao gerar múltiplas perspectivas da questão do usuário, seu objetivo é ajudar o usuário a 
    suerar algumas limitações da busca baseada em similaridade. Forneça essas perguntas alternativas sparadas por linhas.
    Pergunta originaç: {question}
    """
)

In [12]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm, 
    prompt=QUERY_PROMPT
)

template = """Responda a questão baseado somente no seguinte contexto:{context}"""

In [13]:
prompt = ChatPromptTemplate.from_template(template)

In [14]:
chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

In [15]:
chain.invoke(input(""))

O que é isso?


OllamaEmbeddings: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.84s/it]


' A lei geral de proteção de dados é uma lei que fornece diretrizes para a coleta, armazenamento e processamento de informações pessoais. Inclui várias obrigações para os controladores, como cumprir o artigo 7 e garantir o tratamento de dados pessoais apenas com a administração pública ou para uso compartilhado. Além disso, fornece hipóteses legais que podem ser utilizadas para justificar a coleta e armazenamento de informações pessoais, incluindo a necessidade legal ou regulatória.'

In [23]:
chain.invoke(input(""))

Quais os fundamentos da LGPD?


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.44s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.58s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.46s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.68s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.67s/it]


' Como um modelo de linguagem AI, não tenho a capacidade de fornecer uma análise abrangente dos pontos principais da Lei de Proteção de Dados Pessoal (LGPD), mas posso resumir as principais características e princípios do texto que você forneceu. A LGPD é uma lei significativa destinada a proteger os dados pessoais de todos os cidadãos, independentemente da sua formação ou status jurídico. Os principais conceitos subjacentes à Lei incluem diretrizes básicas relacionadas à proteção dos dados pessoais, bem como as formas econômicas e tecnológicas de inovação e desenvolvimento. Além disso, a LGPD traz o conceito de dado pessoal sensível, que é relacionado com características da personalidade do indivíduo ou escolhas pessoais. A Lei também fornece princípios para anonimização de dados e garantir a segurança dos mecanismos técnicos de proteção de dados.'

In [17]:
chain.invoke(input(""))

Quais os 10 princípios da LGPD


OllamaEmbeddings: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.96s/it]


' Compreender a lei de proteção de dados é importante para qualquer organização ou pessoa que colete, processos e armazenar informações pessoais. A Lei Geral de Proteção de Dados Pessoal (LGPD) está em vigor desde 2018 e regula o tratamento de dados pessoais dos cidadãos, incluindo on-line e off-line atividades. Os objetivos da LGDP são proteger os direitos fundamentais dos indivíduos de liberdade e privacidade e o desenvolvimento livre da personalidade.\n\nA LGPD aplica-se a qualquer pessoa, natural ou jurídico, que realizem atividades relacionadas com dados pessoais on-line e/ou offline. A lei inclui várias obrigações para os agentos de tratamento, incluindo a segurança do tratamento de dados, garantir o direito de acesso dos titulares de dados, a transparência e a disponibilidade dos agentos de tratamento.\n\nOs dados pessoais sensíveis são protegidos por LGPD, incluindo informações relacionadas à raça ou etnia, convicção religiosa, opinião política, filiação sindical ou organizacio

In [25]:
chain.invoke(input(""))

Cite os 10 princípios da LGPD?


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.37s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.55s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.60s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.64s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.70s/it]


' A Lei de Proteção dos Dados da Pessoa (LGPD) é uma lei que regula a coleta, uso e divulgação de informações pessoais no Brasil. Trata a protecção de dados pessoais, incluindo a inviolabilidade da intimidade, direitos humanos, dignidade e o livre desenvolvimento da personalidade natural (pessoa física). A LGPD se aplicam a qualquer pessoa que lida com dados pessoais, on-line ou off-line. Os fundamentos da Lei incluem: inviolabilidade da intimidade e honra da imagem; privacidade; Direitos humanos, livre desenvolvimento da personalidade da pessoa natural; Autodeterminação informativa; Livre iniciativa, concorrência e defesa do consumidor; e o Desenvolvimento econômico e tecnológico e inovação. A LGPD aumenta os conceitos de proteção de dados pessoais, que deve ser realizada de forma efetiva e eficaz, aplicando o princípio constitucional da inviolabilidade à privacidade previsto na Constituição, em seu artigo 5o inciso X.'

In [26]:
chain.invoke(input(""))

Organize a resposta em tópicos. Descreva a LGPD em três tópicos.


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.61s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.59s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.46s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.64s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.65s/it]


' Este resumo é útil para entender os princípios e conceitos básicos da Lei de Proteção de Dados do Brasil (LGPD). No entanto, há alguns aspectos-chave que podem precisar de um nível mais aprofundado de análise e interpretação. Por exemplo, o tratamento sensível dos dados pessoais e a anonimização de dados são conceitos complexos que exigem uma compreensão completa dos princípios legais e técnicos subjacentes. Além disso, o resumo não discute quaisquer consequências ou resultados potenciais da aplicação da LGPD em diferentes partes interessadas, como as cidades e os indivíduos. Portanto, é importante entender as implicações práticas do tratamento de dados pessoais, incluindo os riscos potenciais para o livre desenvolvimento da personalidade e dos direitos fundamentais da privacidade e a liberdade.'

In [27]:
chain.invoke(input(""))

A LGPD combate o terrorismo?


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.66s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.67s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.60s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.42s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.60s/it]


' A Lei de Proteção de Dados de 2021 (LGPD) é uma lei brasileira que regula como as pessoas naturais, online ou offline, seus dados pessoais são tratados e protegidos. Ele visa proteger os direitos fundamentais dos cidadãos e promover a liberdade e o desenvolvimento da personalidade pessoal natural (pessoa física). Os princípios-chave da LGPD incluem a inviolabilidade da privacidade, autodeterminação informacional e a respeitosa e os direitos humanos. A lei também requer que os processadores de dados pessoais devem adotar medidas adequadas para proteger os dados e que eles podem ser responsabilizados por quaisquer violações.'

In [28]:
chain.invoke(input(""))

Quais as referências do documento?


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.46s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.42s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.54s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.43s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.68s/it]


' Como um modelo de linguagem AI, não posso fornecer uma opinião pessoal, pois sou apenas um programa de IA projetado para ajudar as pessoas na sua pesquisa. No entanto, posso fornecer informações e resumos relacionados ao tratamento de dados pessoais de acordo com o texto dado.\n\nDe acordo com o texto dado, a LGPD estabelece os principais conceitos do tratamento dos dados pessoais que devem ser respeitados para proteger os direitos fundamentais da privacidade e da liberdade das pessoas. Esses princípios incluem:\n\n1) Inviolabilidade da intimidade, honra da imagem\n2) Respeito à privacidade\n3) Derechos humanos, livre desenvolvimento da personalidade, dignidade e exercício da cidadania pelas pessoas naturais.\n4) Autodeterminação informativa\n5) Livre iniciativa, livre concorrência e defesa do consumidor\n6) Desenvolvimento econômico e tecnológico e inovação\n7) Liberdade de expressão, informação, comunicação e opinião.\n\nO tratamento dos dados pessoais sensívels é relacionado com c

In [29]:
chain.invoke(input(""))

O que é isso?


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.49s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.41s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.60s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.61s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.43s/it]


' Quais são os fundamentos principais relacionados à proteção dos dados pessoais, e como a Lei de Proteção de Dados (LGPD) garante o respeito por esses princípios?'

In [30]:
chain.invoke(input(""))

O que é isso?


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.50s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.61s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.54s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.58s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.54s/it]


' A Lei Geral dos Direitos na Internet (LGPD) é uma lei brasileira que regula a coleta, o processamento e o tratamento de dados pessoais online ou offline. Ele estabelece os fundamentos da proteção de dados pessoais, incluindo direitos humanos e a inviolabilidade da privacidade, e exige que quaisquer órgãos públicas ou privados que lidam com dados pessoais sejam responsáveis por suas segurança e uso. A LGPD se estende a qualquer pessoa que envolva o tratamento de dados pessoais, incluindo cookies, endereças IP, endereças de e-mail e números de telefone. O tratamento de dados sensíveis relacionados com características da personalidade do indivíduo ou escolhas pessoais também são protegidos pela LGPD. A anonimização de dados é uma opção para os usuários que não possam ser identificados, e a segurança dos dados devem estar sob responsabilidade dos órgãos competentes da coordenação do Tecnologia da Informação.'